In [1]:
import model
import model_img
import fastText
from keras.callbacks import ModelCheckpoint, Callback
from keras.models import load_model, Model
from keras.optimizers import *
from keras.layers import *
from keras import regularizers
from importlib import reload
from sklearn import metrics as sklm
import numpy as np
import keras.backend as K

Using TensorFlow backend.


In [2]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [3]:
reload(model_img)
reload(model)

<module 'model' from '/srv/home/gwiedemann/pss-lre/model.py'>

## Text model features

In [4]:
data_text_train = model.read_csv_data("data/archive20k/text/dataset.train")
data_text_test = model.read_csv_data("data/archive20k/text/dataset.test")

In [5]:
if 'ft' not in locals():
    ft = fastText.load_model("./../embeddings/wiki.de.bin")
    model.ft = ft

In [6]:
# model_text = load_model("models/exp1_single-page_repeat-07.hdf5")
model_text = load_model("models/exp1_prev-page_repeat-01.hdf5")

In [7]:
_, y_true, _, _, _ = zip(*data_text_test)
y_true = [1 if y == 'FirstPage' else 0 for y in y_true]
y_predict = np.round(model_text.predict_generator(model.TextFeatureGenerator2(data_text_test)))
print("Accuracy: " + str(sklm.accuracy_score(y_true, y_predict)))
print("Kappa: " + str(sklm.cohen_kappa_score(y_true, y_predict)))

Accuracy: 0.9154072620215898
Kappa: 0.6467880796033505


In [8]:
model_text.layers.pop()
model_text_features = Model(model_text.input, model_text.layers[-1].output)

In [9]:
text_features_train = model_text_features.predict_generator(model.TextFeatureGenerator2(data_text_train))
text_features_test = model_text_features.predict_generator(model.TextFeatureGenerator2(data_text_test))

## Image model features

In [10]:
img_dim = (224,224)

In [11]:
data_image_train = model_img.read_csv_data("data/archive20k/text/dataset.train")
data_image_test = model_img.read_csv_data("data/archive20k/text/dataset.test")

In [12]:
# model_image = load_model("models/exp2_img_repeat-07.hdf5")
model_image = load_model("models/exp2_prev-page_repeat-05.hdf5")

In [13]:
_, y_true, _, _, _ = zip(*data_text_test)
y_true = [1 if y == 'FirstPage' else 0 for y in y_true]
y_predict = np.round(model_image.predict_generator(model_img.ImageFeatureGenerator(data_image_test, img_dim, prevpage=True)))
print("Accuracy: " + str(sklm.accuracy_score(y_true, y_predict)))
print("Kappa: " + str(sklm.cohen_kappa_score(y_true, y_predict)))

Accuracy: 0.8985279685966634
Kappa: 0.5960922016537417


In [14]:
model_image.layers.pop()
model_image_features = Model(model_image.input, model_image.layers[-1].output)

In [15]:
image_features_train = model_image_features.predict_generator(model_img.ImageFeatureGenerator(data_image_train, img_dim, prevpage=True))
image_features_test = model_image_features.predict_generator(model_img.ImageFeatureGenerator(data_image_test, img_dim, prevpage=True))

# Training and test targets

In [16]:
# Training data
_, data_train_y, _, _, _ = zip(*data_text_train)
data_train_y = [1 if y == 'FirstPage' else 0 for y in data_train_y]
# Test data
_, data_test_y, _, _, _ = zip(*data_text_test)
data_test_y = [1 if y == 'FirstPage' else 0 for y in data_test_y]

In [17]:
class ValidationCheckpoint(Callback):
    def __init__(self, filepath, validation_x, validation_y, metric = 'kappa'):
        self.metric = metric
        self.max_metric = float('-inf')
        self.max_metrics = None
        self.filepath = filepath
        self.history = []
        self.validation_x = validation_x
        self.validation_y = validation_y

    def on_epoch_end(self, epoch, logs={}):
        
        predicted_labels = np.round(self.model.predict(self.validation_x))
        true_labels = self.validation_y

        eval_metrics = {
            'accuracy' : sklm.accuracy_score(true_labels, predicted_labels),
            'f1_micro' : sklm.f1_score(true_labels, predicted_labels, average='micro'),
            'f1_macro' : sklm.f1_score(true_labels, predicted_labels, average='macro'),
            'f1_binary' : sklm.f1_score(true_labels, predicted_labels, average='binary', pos_label = 1),
            'kappa' : sklm.cohen_kappa_score(true_labels, predicted_labels)
        }
        eval_metric = eval_metrics[self.metric]
        self.history.append(eval_metric)
        
        if epoch > -1 and eval_metric > self.max_metric:
            print("\n" + self.metric + " improvement: " + str(eval_metric) + " (before: " + str(self.max_metric) + "), saving to " + self.filepath)
            self.max_metric = eval_metric     # optimization target
            self.max_metrics = eval_metrics   # all metrics
            self.model.save(self.filepath)

## LDA Features

In [18]:
lda_train_x = []
with open("data/archive20k/lda_train.csv") as f:
    next(f)
    for l in f:
        lda_train_x.append([float(n) for n in l.split(",")])
lda_train_x = np.array(lda_train_x)
print(lda_train_x.shape)

lda_test_x = []
with open("data/archive20k/lda_test.csv") as f:
    next(f)
    for l in f:
        lda_test_x.append([float(n) for n in l.split(",")])
lda_test_x = np.array(lda_test_x)
print(lda_test_x.shape)

(17376, 102)
(5095, 102)


In [ ]:
features_x_train = lda_train_x # [:,-2:]
features_x_test = lda_test_x # [:,-2:]
sequence_x_train = np.empty((len(features_x_train),2,len(features_x_train[0])))
sep_tp_x_train = np.empty((len(features_x_train),len(features_x_train[0])))
sep_pp_x_train = np.empty((len(features_x_train),len(features_x_train[0])))
for i, d in enumerate(features_x_train):
    if d[3] == "":
        prev_page = np.zeros((1,len(features_x_train[0])))
    else:
        prev_page = features_x_train[i-1]
    sequence_x_train[i][0] = sep_pp_x_train[i] = prev_page
    sequence_x_train[i][1] = sep_tp_x_train[i] = features_x_train[i]

sequence_x_test = np.empty((len(features_x_test),2,len(features_x_test[0])))
sep_tp_x_test = np.empty((len(features_x_test),len(features_x_test[0])))
sep_pp_x_test = np.empty((len(features_x_test),len(features_x_test[0])))
for i, d in enumerate(features_x_test):
    if d[3] == "":
        prev_page = np.zeros((1,len(features_x_test[0])))
    else:
        prev_page = features_x_test[i-1]
    sequence_x_test[i][0] = sep_pp_x_test[i] = prev_page
    sequence_x_test[i][1] = sep_tp_x_test[i] = features_x_test[i]
print(sep_tp_x_train.shape)
print(sep_pp_x_train.shape)
print(sep_tp_x_test.shape)
print(sep_pp_x_test.shape)

In [20]:
sequence_x_2inputs_train = [sep_tp_x_train, sep_pp_x_train]
sequence_x_2inputs_test = [sep_tp_x_test, sep_pp_x_test]

In [73]:
input_tp = Input(shape=sequence_x_2inputs_train[0][0].shape)
input_pp = Input(shape=sequence_x_2inputs_train[1][0].shape)
difference = subtract([input_pp, input_tp])
final_feat = concatenate([input_tp, input_pp, difference])
final_feat = Dense(400)(final_feat)
final_feat = LeakyReLU()(final_feat)
final_feat = Dropout(0.7)(final_feat)
model_output = Dense(1, activation = 'sigmoid')(final_feat)
combined_model = Model([input_tp, input_pp], model_output)
model_path = "exp3_img-text_ldadiff_model.hdf5"
checkpoint = ValidationCheckpoint(model_path, sequence_x_2inputs_test, data_test_y)
combined_model.compile(loss = 'binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
combined_model.fit(sequence_x_2inputs_train, data_train_y, validation_data = (sequence_x_2inputs_test, data_test_y), 
                   batch_size=4096, epochs=40, callbacks=[checkpoint])


Train on 17376 samples, validate on 5095 samples
Epoch 1/40
17376/17376 [==============================] - 3s 189us/step - loss: 0.6127 - acc: 0.7385 - val_loss: 0.5008 - val_acc: 0.8563

kappa improvement: 0.006965412572889185 (before: -inf), saving to exp3_img-text_lda_model.hdf5
Epoch 2/40
17376/17376 [==============================] - 0s 5us/step - loss: 0.4899 - acc: 0.8239 - val_loss: 0.4228 - val_acc: 0.8569

kappa improvement: 0.017672222846677133 (before: 0.006965412572889185), saving to exp3_img-text_lda_model.hdf5
Epoch 3/40
17376/17376 [==============================] - 0s 4us/step - loss: 0.4254 - acc: 0.8309 - val_loss: 0.3781 - val_acc: 0.8606

kappa improvement: 0.08284316211145482 (before: 0.017672222846677133), saving to exp3_img-text_lda_model.hdf5
Epoch 4/40
17376/17376 [==============================] - 0s 4us/step - loss: 0.3792 - acc: 0.8414 - val_loss: 0.3489 - val_acc: 0.8683

kappa improvement: 0.18911665470846706 (before: 0.08284316211145482), saving to exp3_

# Late Fusion

## Power Weighting

In [1]:
model_text = load_model("models/exp1_prev-page_repeat-01.hdf5")
model_image = load_model("models/exp2_prev-page_repeat-05.hdf5")
model_lda = load_model("models/exp3_img-text_lda_model.hdf5")

NameError: name 'load_model' is not defined

In [42]:
p_t_train = model_text.predict_generator(model.TextFeatureGenerator2(data_text_train))
p_v_train = model_image.predict_generator(model_img.ImageFeatureGenerator(data_image_train, img_dim, prevpage=True))
p_l_train = model_lda.predict(sequence_x_2inputs_train)
p_t_train = np.concatenate([1 - p_t_train, p_t_train], axis = 1) # probability from text model
p_v_train = np.concatenate([1 - p_v_train, p_v_train], axis = 1) # probability from visual model
p_l_train = np.concatenate([1 - p_l_train, p_l_train], axis = 1) # probability from lda model

In [43]:
p_t_test = model_text.predict_generator(model.TextFeatureGenerator2(data_text_test))
p_v_test = model_image.predict_generator(model_img.ImageFeatureGenerator(data_image_test, img_dim, prevpage=True))
p_l_test = model_lda.predict(sequence_x_2inputs_test)
p_t_test = np.concatenate([1 - p_t_test, p_t_test], axis = 1) # probability from text model
p_v_test = np.concatenate([1 - p_v_test, p_v_test], axis = 1) # probability from visual model
p_l_test = np.concatenate([1 - p_l_test, p_l_test], axis = 1) # probability from lda model

In [48]:
# scoring with diffent i, j in (0,1] power normalizations
max_kappa = 0
test_exponents = [x / 10 for x in range(1,11)]
for i in test_exponents:
    for j in test_exponents:
        for k in test_exponents:
            y_predict = np.argmax(np.power(p_t_train, i) * np.power(p_v_train, j) * np.power(p_l_train, k), axis = 1)
            acc = sklm.accuracy_score(data_train_y, y_predict)
            kappa = sklm.cohen_kappa_score(data_train_y, y_predict)
            if kappa > max_kappa:
                max_kappa = kappa
                print(str(i) + " " + str(j) + " " + str(k))
                print("Accuracy: " + str(acc))
                print("Kappa: " + str(kappa))

0.1 0.1
Accuracy: 0.9236506378802748
Kappa: 0.67242172941123
0.2 0.1
Accuracy: 0.9273797841020608
Kappa: 0.6874854945127814
0.3 0.1
Accuracy: 0.9295387634936212
Kappa: 0.6965941370887814
0.9 0.2
Accuracy: 0.929342492639843
Kappa: 0.6973866561077015


In [2]:
i = 0.1
j = 0.2
k = 0.1 
y_predict = np.argmax(np.power(p_t_test, i) * np.power(p_v_test, j) * np.power(p_l_test, k), axis = 1)
acc = sklm.accuracy_score(y_true, y_predict)
kappa = sklm.cohen_kappa_score(y_true, y_predict)
print(str(i) + " " + str(j) + " " + str(k))
print("Accuracy: " + str(acc))
print("Kappa: " + str(kappa))

NameError: name 'np' is not defined

*Test set*
.......
Best results: i = 0.9 j = 0.2 without LDA
Accuracy: 0.929342492639843
Kappa: 0.6973866561077015

Best results: i = 0.4 j = 0.1 k = 0.2 with LDA
Accuracy: 0.9336604514229637
Kappa: 0.7078421226747195

In [47]:
from matplotlib.pyplot import imshow, figure
from keras.preprocessing.image import load_img
i = 0.4 
j = 0.1 
k = 0.2
n = 0
y_predict = np.argmax(np.power(a_t, i) * np.power(a_v, j) * np.power(a_l, k), axis = 1)
for i_y, y in enumerate(y_predict):
    if y==0 and y_true[i_y] == 1:
        img_path = 'data/archive20k/images/%s.png' % data_text_test[i_y][0]
        print(img_path)
#         image = load_img(img_path)
#         figure()
#         imshow(image)
        n += 1
print(n) # 240 FN (handwriting,...), 98 FP (real 1st pages, tables/figures/complex layoot)

data/archive20k/images/189_2_0002.png
data/archive20k/images/189_2_0003.png
data/archive20k/images/189_2_0004.png
data/archive20k/images/189_2_0025.png
data/archive20k/images/189_2_0038.png
data/archive20k/images/189_2_0043.png
data/archive20k/images/189_2_0043_001.png
data/archive20k/images/189_2_0046.png
data/archive20k/images/189_2_0052.png
data/archive20k/images/189_2_0053.png
data/archive20k/images/189_2_0055.png
data/archive20k/images/189_2_0063.png
data/archive20k/images/189_2_0079.png
data/archive20k/images/189_2_0113.png
data/archive20k/images/189_2_0118.png
data/archive20k/images/189_2_0122.png
data/archive20k/images/189_2_0131.png
data/archive20k/images/189_2_0132.png
data/archive20k/images/189_2_0170.png
data/archive20k/images/189_2_0174.png
data/archive20k/images/189_2_0217.png
data/archive20k/images/189_2_0250.png
data/archive20k/images/189_2_0256.png
data/archive20k/images/189_2_0257.png
data/archive20k/images/189_2_0258.png
data/archive20k/images/63_0015.png
data/archiv

In [28]:
y_predict

array([1, 0, 0, ..., 0, 0, 0])

## Logistic Regression

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
pred_t = model_text.predict_generator(model.TextFeatureGenerator2(data_text_train))
pred_v = model_image.predict_generator(model_img.ImageFeatureGenerator(data_image_train, img_dim, prevpage=True))
pred_l = model_lda.predict(sequence_x_2inputs_train)

NameError: name 'model_lda' is not defined

In [34]:
lr_model = LogisticRegression(C=1, max_iter=1000)
X = np.column_stack([pred_t, pred_v, pred_l])
lr_model.fit(X, data_train_y)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [36]:
y_predict = lr_model.predict(np.column_stack([p_t, p_v, p_l]))
acc = sklm.accuracy_score(y_true, y_predict)
kappa = sklm.cohen_kappa_score(y_true, y_predict)
print(acc)
print(kappa)

NameError: name 'p_t' is not defined

In [25]:
from sklearn.svm import SVC
svm_model = SVC(C=1)
X = np.column_stack([pred_t, pred_v, pred_l])
svm_model.fit(X, data_train_y)
y_predict = svm_model.predict(np.column_stack([p_t, p_v, p_l]))
acc = sklm.accuracy_score(y_true, y_predict)
kappa = sklm.cohen_kappa_score(y_true, y_predict)
print(acc)
print(kappa)

0.9050049067713445
0.613494874768816


# Early Fusion

In [25]:
# with LDA
# features_x_train = np.concatenate([text_features_train, lda_train_x], axis = 1)
# features_x_test = np.concatenate([text_features_test, lda_test_x], axis = 1)

# without LDA
features_x_train = text_features_train
features_x_test = text_features_test

In [26]:
data_train_x = [features_x_train, image_features_train]
data_test_x = [features_x_test, image_features_test]
print(len(data_train_x[0][0]))
print(len(data_train_x[1][0]))

256
256


In [31]:
n_repeats = 10
all_acc = []
all_kap = []
for n_repeat in range(n_repeats):
    text_input = Input(shape=(len(data_train_x[0][0]),))
    img_input = Input(shape=(len(data_train_x[1][0]),))
    combined = concatenate([text_input, img_input])
    combined = Dense(400)(combined)
    combined = LeakyReLU()(combined)
    combined = Dropout(0.9)(combined)
    model_output = Dense(1, activation = 'sigmoid')(combined)
    combined_model = Model([text_input, img_input], model_output)

    model_path = "exp3_img-text_mlp_simple.hdf5"
    checkpoint = ValidationCheckpoint(model_path, data_test_x, data_test_y)
    combined_model.compile(loss = 'binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

    combined_model.fit(data_train_x, data_train_y, 
                       validation_data = (data_test_x, data_test_y), 
                       batch_size=32, epochs=25, 
                       callbacks=[checkpoint]
                      )

    combined_model.load_weights(model_path)
    y_predict = np.round(combined_model.predict(data_test_x))
    all_acc.append(sklm.accuracy_score(data_test_y, y_predict))
    all_kap.append(sklm.cohen_kappa_score(data_test_y, y_predict))
    print("Accuracy: " + str(all_acc[-1]))
    print("Kappa: " + str(all_kap[-1]))
print(np.average(np.array(all_acc)))
print(np.average(np.array(all_kap)))

Train on 17376 samples, validate on 5095 samples
Epoch 1/25
17376/17376 [==============================] - 6s 324us/step - loss: 0.0299 - acc: 0.9883 - val_loss: 1.2010 - val_acc: 0.9083

kappa improvement: 0.6473094610769567 (before: -inf), saving to exp3_img-text_mlp_simple.hdf5
Epoch 2/25
17376/17376 [==============================] - 3s 155us/step - loss: 0.0254 - acc: 0.9923 - val_loss: 1.1908 - val_acc: 0.9168

kappa improvement: 0.6644590102823771 (before: 0.6473094610769567), saving to exp3_img-text_mlp_simple.hdf5
Epoch 3/25
17376/17376 [==============================] - 3s 150us/step - loss: 0.0345 - acc: 0.9911 - val_loss: 1.2101 - val_acc: 0.9195

kappa improvement: 0.6737009817321284 (before: 0.6644590102823771), saving to exp3_img-text_mlp_simple.hdf5
Epoch 4/25
17376/17376 [==============================] - 3s 151us/step - loss: 0.0308 - acc: 0.9942 - val_loss: 1.2383 - val_acc: 0.9170
Epoch 5/25
17376/17376 [==============================] - 3s 152us/step - loss: 0.0285

17376/17376 [==============================] - 3s 156us/step - loss: 0.0261 - acc: 0.9932 - val_loss: 1.2191 - val_acc: 0.9189
Epoch 5/25
17376/17376 [==============================] - 3s 162us/step - loss: 0.0315 - acc: 0.9937 - val_loss: 1.2453 - val_acc: 0.9184
Epoch 6/25
17376/17376 [==============================] - 3s 154us/step - loss: 0.0271 - acc: 0.9939 - val_loss: 1.2093 - val_acc: 0.9227

kappa improvement: 0.6813830758120452 (before: 0.6702239961781182), saving to exp3_img-text_mlp_simple.hdf5
Epoch 7/25
17376/17376 [==============================] - 3s 151us/step - loss: 0.0278 - acc: 0.9938 - val_loss: 1.3177 - val_acc: 0.9136
Epoch 8/25
17376/17376 [==============================] - 3s 150us/step - loss: 0.0282 - acc: 0.9935 - val_loss: 1.2154 - val_acc: 0.9229
Epoch 9/25
17376/17376 [==============================] - 3s 148us/step - loss: 0.0375 - acc: 0.9932 - val_loss: 1.2543 - val_acc: 0.9195
Epoch 10/25
17376/17376 [==============================] - 3s 155us/step -

17376/17376 [==============================] - 3s 148us/step - loss: 0.0307 - acc: 0.9931 - val_loss: 1.2488 - val_acc: 0.9199
Epoch 9/25
17376/17376 [==============================] - 3s 152us/step - loss: 0.0318 - acc: 0.9933 - val_loss: 1.2359 - val_acc: 0.9215
Epoch 10/25
17376/17376 [==============================] - 3s 149us/step - loss: 0.0380 - acc: 0.9926 - val_loss: 1.2878 - val_acc: 0.9180
Epoch 11/25
17376/17376 [==============================] - 3s 154us/step - loss: 0.0325 - acc: 0.9914 - val_loss: 1.3876 - val_acc: 0.9113
Epoch 12/25
17376/17376 [==============================] - 3s 156us/step - loss: 0.0313 - acc: 0.9913 - val_loss: 1.2282 - val_acc: 0.9223

kappa improvement: 0.6830482115085537 (before: 0.6797351744563869), saving to exp3_img-text_mlp_simple.hdf5
Epoch 13/25
17376/17376 [==============================] - 3s 149us/step - loss: 0.0241 - acc: 0.9949 - val_loss: 1.2060 - val_acc: 0.9238
Epoch 14/25
17376/17376 [==============================] - 3s 162us/st

In [32]:
combined_model.load_weights(model_path)
y_predict = np.round(combined_model.predict(data_test_x))
print("Accuracy: " + str(sklm.accuracy_score(data_test_y, y_predict)))
print("Kappa: " + str(sklm.cohen_kappa_score(data_test_y, y_predict)))

Accuracy: 0.9224730127576055
Kappa: 0.6777936544149725


Early fusion with LDA

Accuracy: 0.9228263002944063
Kappa: 0.679234515403952

Early fusion without LDA

Accuracy: 0.9235328753680079
Kappa: 0.6803052115566901

--

extract 10 fp and 10 fn files


# MLP: predecessor page
* extract features from text and image
* combine 2 in sequence RNN

In [134]:
features_x_train = np.concatenate([text_features_train, image_features_train, lda_train_x], axis = 1)
features_x_test = np.concatenate([text_features_test, image_features_test, lda_test_x], axis = 1)

In [135]:
sequence_x_train = np.empty((len(features_x_train),2,len(features_x_train[0])))
sep_tp_x_train = np.empty((len(features_x_train),len(features_x_train[0])))
sep_pp_x_train = np.empty((len(features_x_train),len(features_x_train[0])))
for i, d in enumerate(features_x_train):
    if d[3] == "":
        prev_page = np.zeros((1,len(features_x_train[0])))
    else:
        prev_page = features_x_train[i-1]
    sequence_x_train[i][0] = sep_pp_x_train[i] = prev_page
    sequence_x_train[i][1] = sep_tp_x_train[i] = features_x_train[i]

sequence_x_test = np.empty((len(features_x_test),2,len(features_x_test[0])))
sep_tp_x_test = np.empty((len(features_x_test),len(features_x_test[0])))
sep_pp_x_test = np.empty((len(features_x_test),len(features_x_test[0])))
for i, d in enumerate(features_x_test):
    if d[3] == "":
        prev_page = np.zeros((1,len(features_x_test[0])))
    else:
        prev_page = features_x_test[i-1]
    sequence_x_test[i][0] = sep_pp_x_test[i] = prev_page
    sequence_x_test[i][1] = sep_tp_x_test[i] = features_x_test[i]

In [136]:
print(sep_tp_x_train.shape)
print(sep_pp_x_train.shape)
print(sep_tp_x_test.shape)
print(sep_pp_x_test.shape)

(17376, 614)
(17376, 614)
(5095, 614)
(5095, 614)


In [137]:
sequence_x_train.shape

(17376, 2, 614)

In [497]:
model_path = "exp3_img-text_rnn-model.hdf5"
checkpoint = ValidationCheckpoint(model_path, sequence_x_test, data_test_y)

sgd = SGD(lr=1e-3, decay=1e-6, nesterov=True)

rnn_input = Input(shape=sequence_x_train[0].shape,)
#attention_probs = Dense(sequence_x_train[1][0].shape[0], activation='softmax')(rnn_input)
#attention_mul = multiply([rnn_input, attention_probs])
rnn_block = Bidirectional(GRU(300, return_sequences = True))(rnn_input)
rnn_block = Bidirectional(GRU(300))(rnn_block)
combined = Dense(128)(rnn_block)
combined = LeakyReLU()(combined)
model_output = Dense(1, activation = 'sigmoid')(combined)
combined_model = Model([rnn_input], model_output)
combined_model.compile(loss = 'binary_crossentropy', optimizer=Nadam(lr=1e-5), metrics=['accuracy'])
combined_model.fit(sequence_x_train, data_train_y, validation_data = (sequence_x_test, data_test_y), 
                   batch_size=32, epochs=2, callbacks=[checkpoint])

Train on 17376 samples, validate on 5095 samples
Epoch 1/2
17376/17376 [==============================] - 30s 2ms/step - loss: 0.1479 - acc: 0.9483 - val_loss: 0.2366 - val_acc: 0.9123

kappa improvement: 0.6328194338641613 (before: -inf), saving to exp3_img-text_rnn-model.hdf5
Epoch 2/2
17376/17376 [==============================] - 13s 759us/step - loss: 0.0349 - acc: 0.9899 - val_loss: 0.2938 - val_acc: 0.9142

kappa improvement: 0.6401926285638102 (before: 0.6328194338641613), saving to exp3_img-text_rnn-model.hdf5


In [498]:
feature_input = Input(shape=sequence_x_train[0].shape)
combined = Flatten()(feature_input)
combined = Dropout(0.25)(combined)
combined = Dense(300, activation = 'relu')(combined)
model_output = Dense(1, activation = 'sigmoid')(combined)
combined_model = Model([feature_input], model_output)
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model_path = "exp3_img-text_mlp_model.hdf5"
checkpoint = ValidationCheckpoint(model_path, sequence_x_test, data_test_y)
combined_model.compile(loss = 'binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
combined_model.fit(sequence_x_train, data_train_y, validation_data = (sequence_x_test, data_test_y), 
                   batch_size=256, epochs=10, callbacks=[checkpoint])

Train on 17376 samples, validate on 5095 samples
Epoch 1/10
17376/17376 [==============================] - 14s 832us/step - loss: 0.1191 - acc: 0.9523 - val_loss: 0.2363 - val_acc: 0.9191

kappa improvement: 0.6625556609384795 (before: -inf), saving to exp3_img-text_mlp_model.hdf5
Epoch 2/10
17376/17376 [==============================] - 1s 29us/step - loss: 0.0581 - acc: 0.9810 - val_loss: 0.2520 - val_acc: 0.9184
Epoch 3/10
17376/17376 [==============================] - 0s 28us/step - loss: 0.0462 - acc: 0.9852 - val_loss: 0.2651 - val_acc: 0.9178
Epoch 4/10
17376/17376 [==============================] - 0s 27us/step - loss: 0.0395 - acc: 0.9880 - val_loss: 0.2783 - val_acc: 0.9178
Epoch 5/10
17376/17376 [==============================] - 1s 29us/step - loss: 0.0352 - acc: 0.9904 - val_loss: 0.2907 - val_acc: 0.9160
Epoch 6/10
17376/17376 [==============================] - 0s 25us/step - loss: 0.0316 - acc: 0.9904 - val_loss: 0.3020 - val_acc: 0.9148
Epoch 7/10
17376/17376 [=========

In [141]:
sequence_x_2inputs_train = [sep_tp_x_train, sep_pp_x_train]
sequence_x_2inputs_test = [sep_tp_x_test, sep_pp_x_test]

In [142]:
sequence_x_2inputs_train[0].shape

(17376, 614)

In [208]:
n_repeats = 10
all_res = []
for n_repeat in range(n_repeats):
    input_tp = Input(shape=sequence_x_2inputs_train[0][0].shape)
    input_pp = Input(shape=sequence_x_2inputs_train[1][0].shape)
    # similarity = dot([input_tp, input_pp], axes=1, normalize=True)
    difference = subtract([input_pp, input_tp])
    final_feat = concatenate([input_tp, input_pp, difference]) 
    final_feat = Dense(500)(final_feat)
    final_feat = LeakyReLU()(final_feat)
    final_feat = Dropout(0.75)(final_feat)
    model_output = Dense(1, activation = 'sigmoid')(final_feat)
    combined_model = Model([input_tp, input_pp], model_output)
    # sgd = SGD(lr=0.001, decay=1e-5, momentum=0.9, nesterov=True)
    model_path = "exp3_img-text_mlp_model.hdf5"
    checkpoint = ValidationCheckpoint(model_path, sequence_x_2inputs_test, data_test_y)
    combined_model.compile(loss = 'binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    combined_model.fit(sequence_x_2inputs_train, data_train_y, validation_data = (sequence_x_2inputs_test, data_test_y), 
                       batch_size=4096, epochs=5, callbacks=[checkpoint])
    all_res.append(checkpoint.max_metric)
np.average(all_res)
# 0.689497014044883 (with: pp, difference, similarity)
# 0.6896352389170874 (with: pp, difference)
# 0.6638292194420565 (with: pp)
# 0.6787854658268276 (with: difference)
# 0.6801960159877674 (with: tp only)

Train on 17376 samples, validate on 5095 samples
Epoch 1/5
17376/17376 [==============================] - 8s 464us/step - loss: 0.5035 - acc: 0.9069 - val_loss: 0.9685 - val_acc: 0.9237

kappa improvement: 0.6834228485402698 (before: -inf), saving to exp3_img-text_mlp_model.hdf5
Epoch 2/5
17376/17376 [==============================] - 0s 16us/step - loss: 0.0890 - acc: 0.9885 - val_loss: 0.9710 - val_acc: 0.9258
Epoch 3/5
17376/17376 [==============================] - 0s 14us/step - loss: 0.0703 - acc: 0.9908 - val_loss: 0.9727 - val_acc: 0.9254

kappa improvement: 0.6850948245014801 (before: 0.6834228485402698), saving to exp3_img-text_mlp_model.hdf5
Epoch 4/5
17376/17376 [==============================] - 0s 16us/step - loss: 0.0491 - acc: 0.9939 - val_loss: 0.9776 - val_acc: 0.9252
Epoch 5/5
17376/17376 [==============================] - 0s 16us/step - loss: 0.0410 - acc: 0.9944 - val_loss: 0.9771 - val_acc: 0.9252
Train on 17376 samples, validate on 5095 samples
Epoch 1/5
17376/173

0.6887521905287414

# Hyperparameters

In [211]:
n_repeats = 10
n_dense = [300,400,500]
n_dropouts = [i / 10 for i in range(1,10)]
param_selection_results = {}
for p_dense in n_dense:
    for p_dropout in n_dropouts:
        exp_identifier = str(p_dense) + '_' + str(p_dropout)
        repeat_res = []
        print(exp_identifier)
        print("=================================")
        for n_repeat in range(n_repeats):
            input_tp = Input(shape=sequence_x_2inputs_train[0][0].shape)
            input_pp = Input(shape=sequence_x_2inputs_train[1][0].shape)
            difference = subtract([input_pp, input_tp])
            final_feat = concatenate([input_tp, input_pp, difference])
            final_feat = Dense(p_dense)(final_feat)
            final_feat = LeakyReLU()(final_feat)
            final_feat = Dropout(p_dropout)(final_feat)
            model_output = Dense(1, activation = 'sigmoid')(final_feat)
            combined_model = Model([input_tp, input_pp], model_output)
            model_path = "exp3_img-text_mlp_model.hdf5"
            checkpoint = ValidationCheckpoint(model_path, sequence_x_2inputs_test, data_test_y)
            combined_model.compile(loss = 'binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
            combined_model.fit(sequence_x_2inputs_train, data_train_y, validation_data = (sequence_x_2inputs_test, data_test_y), 
                               batch_size=4096, epochs=10, callbacks=[checkpoint])
            
            
            repeat_res.append(checkpoint.max_metric)
        param_selection_results[exp_identifier] = np.average(repeat_res)

300_0.1
Train on 17376 samples, validate on 5095 samples
Epoch 1/10
17376/17376 [==============================] - 9s 517us/step - loss: 0.4939 - acc: 0.8426 - val_loss: 0.8531 - val_acc: 0.9262

kappa improvement: 0.6804976692600839 (before: -inf), saving to exp3_img-text_mlp_model.hdf5
Epoch 2/10
17376/17376 [==============================] - 0s 19us/step - loss: 0.0571 - acc: 0.9904 - val_loss: 0.8879 - val_acc: 0.9286

kappa improvement: 0.6936578597444644 (before: 0.6804976692600839), saving to exp3_img-text_mlp_model.hdf5
Epoch 3/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0478 - acc: 0.9927 - val_loss: 0.9101 - val_acc: 0.9282
Epoch 4/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0411 - acc: 0.9938 - val_loss: 0.9280 - val_acc: 0.9270
Epoch 5/10
17376/17376 [==============================] - 0s 15us/step - loss: 0.0341 - acc: 0.9947 - val_loss: 0.9331 - val_acc: 0.9248
Epoch 6/10
17376/17376 [==============================]

17376/17376 [==============================] - 0s 15us/step - loss: 0.0186 - acc: 0.9968 - val_loss: 0.9221 - val_acc: 0.9237
Epoch 9/10
17376/17376 [==============================] - 0s 15us/step - loss: 0.0158 - acc: 0.9969 - val_loss: 0.9185 - val_acc: 0.9229
Epoch 10/10
17376/17376 [==============================] - 0s 14us/step - loss: 0.0128 - acc: 0.9975 - val_loss: 0.9163 - val_acc: 0.9221
300_0.2
Train on 17376 samples, validate on 5095 samples
Epoch 1/10
17376/17376 [==============================] - 11s 623us/step - loss: 0.8117 - acc: 0.7989 - val_loss: 0.8270 - val_acc: 0.9268

kappa improvement: 0.6797554903042891 (before: -inf), saving to exp3_img-text_mlp_model.hdf5
Epoch 2/10
17376/17376 [==============================] - 0s 19us/step - loss: 0.0558 - acc: 0.9903 - val_loss: 0.8826 - val_acc: 0.9248
Epoch 3/10
17376/17376 [==============================] - 0s 19us/step - loss: 0.0477 - acc: 0.9920 - val_loss: 0.8907 - val_acc: 0.9264

kappa improvement: 0.6842111787383

17376/17376 [==============================] - 0s 15us/step - loss: 0.0083 - acc: 0.9983 - val_loss: 0.9055 - val_acc: 0.9203
Epoch 10/10
17376/17376 [==============================] - 0s 14us/step - loss: 0.0061 - acc: 0.9986 - val_loss: 0.9042 - val_acc: 0.9197
300_0.3
Train on 17376 samples, validate on 5095 samples
Epoch 1/10
17376/17376 [==============================] - 11s 613us/step - loss: 0.3106 - acc: 0.9266 - val_loss: 0.8718 - val_acc: 0.9225

kappa improvement: 0.6800221000624049 (before: -inf), saving to exp3_img-text_mlp_model.hdf5
Epoch 2/10
17376/17376 [==============================] - 0s 21us/step - loss: 0.0716 - acc: 0.9890 - val_loss: 0.8817 - val_acc: 0.9244

kappa improvement: 0.687762890331207 (before: 0.6800221000624049), saving to exp3_img-text_mlp_model.hdf5
Epoch 3/10
17376/17376 [==============================] - 0s 15us/step - loss: 0.0488 - acc: 0.9922 - val_loss: 0.8885 - val_acc: 0.9274
Epoch 4/10
17376/17376 [==============================] - 0s 14us

Epoch 10/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0084 - acc: 0.9984 - val_loss: 0.9425 - val_acc: 0.9217
300_0.4
Train on 17376 samples, validate on 5095 samples
Epoch 1/10
17376/17376 [==============================] - 10s 582us/step - loss: 0.9791 - acc: 0.7940 - val_loss: 0.8328 - val_acc: 0.9268

kappa improvement: 0.6809249379834874 (before: -inf), saving to exp3_img-text_mlp_model.hdf5
Epoch 2/10
17376/17376 [==============================] - 0s 15us/step - loss: 0.0523 - acc: 0.9902 - val_loss: 0.8733 - val_acc: 0.9282

kappa improvement: 0.6938080851203573 (before: 0.6809249379834874), saving to exp3_img-text_mlp_model.hdf5
Epoch 3/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0505 - acc: 0.9912 - val_loss: 0.8937 - val_acc: 0.9280
Epoch 4/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0386 - acc: 0.9937 - val_loss: 0.9048 - val_acc: 0.9270
Epoch 5/10
17376/17376 [=============================

17376/17376 [==============================] - 0s 16us/step - loss: 0.0102 - acc: 0.9976 - val_loss: 0.9127 - val_acc: 0.9211
300_0.5
Train on 17376 samples, validate on 5095 samples
Epoch 1/10
17376/17376 [==============================] - 10s 594us/step - loss: 0.6137 - acc: 0.8570 - val_loss: 0.8423 - val_acc: 0.9242

kappa improvement: 0.6642876651531187 (before: -inf), saving to exp3_img-text_mlp_model.hdf5
Epoch 2/10
17376/17376 [==============================] - 0s 17us/step - loss: 0.0591 - acc: 0.9892 - val_loss: 0.8791 - val_acc: 0.9242

kappa improvement: 0.6723963885927505 (before: 0.6642876651531187), saving to exp3_img-text_mlp_model.hdf5
Epoch 3/10
17376/17376 [==============================] - 0s 18us/step - loss: 0.0449 - acc: 0.9921 - val_loss: 0.9042 - val_acc: 0.9244

kappa improvement: 0.6750112867752129 (before: 0.6723963885927505), saving to exp3_img-text_mlp_model.hdf5
Epoch 4/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0408 - acc: 0

...

Epoch 9/10
17376/17376 [==============================] - 0s 15us/step - loss: 0.0175 - acc: 0.9975 - val_loss: 0.9838 - val_acc: 0.9219
Epoch 10/10
17376/17376 [==============================] - 0s 18us/step - loss: 0.0158 - acc: 0.9977 - val_loss: 0.9926 - val_acc: 0.9215
Train on 17376 samples, validate on 5095 samples
Epoch 1/10
17376/17376 [==============================] - 20s 1ms/step - loss: 0.2817 - acc: 0.9219 - val_loss: 0.9408 - val_acc: 0.9233

kappa improvement: 0.6691517232502813 (before: -inf), saving to exp3_img-text_mlp_model.hdf5
Epoch 2/10
17376/17376 [==============================] - 0s 18us/step - loss: 0.0623 - acc: 0.9911 - val_loss: 0.9620 - val_acc: 0.9258

kappa improvement: 0.6814940380703525 (before: 0.6691517232502813), saving to exp3_img-text_mlp_model.hdf5
Epoch 3/10
17376/17376 [==============================] - 0s 17us/step - loss: 0.0487 - acc: 0.9934 - val_loss: 0.9832 - val_acc: 0.9246
Epoch 4/10
17376/17376 [==============================] - 0s 16

17376/17376 [==============================] - 0s 16us/step - loss: 0.0303 - acc: 0.9956 - val_loss: 1.0254 - val_acc: 0.9223
Epoch 7/10
17376/17376 [==============================] - 0s 17us/step - loss: 0.0245 - acc: 0.9961 - val_loss: 1.0204 - val_acc: 0.9223
Epoch 8/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0197 - acc: 0.9970 - val_loss: 1.0162 - val_acc: 0.9199
Epoch 9/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0157 - acc: 0.9972 - val_loss: 1.0196 - val_acc: 0.9207
Epoch 10/10
17376/17376 [==============================] - 0s 17us/step - loss: 0.0116 - acc: 0.9982 - val_loss: 1.0205 - val_acc: 0.9205
Train on 17376 samples, validate on 5095 samples
Epoch 1/10
17376/17376 [==============================] - 20s 1ms/step - loss: 0.2549 - acc: 0.9276 - val_loss: 0.9139 - val_acc: 0.9254

kappa improvement: 0.6865732046882083 (before: -inf), saving to exp3_img-text_mlp_model.hdf5
Epoch 2/10
17376/17376 [=====================

17376/17376 [==============================] - 0s 17us/step - loss: 0.0468 - acc: 0.9938 - val_loss: 1.0060 - val_acc: 0.9229

kappa improvement: 0.6690533153177722 (before: 0.6675230083464476), saving to exp3_img-text_mlp_model.hdf5
Epoch 5/10
17376/17376 [==============================] - 0s 17us/step - loss: 0.0361 - acc: 0.9949 - val_loss: 1.0073 - val_acc: 0.9219
Epoch 6/10
17376/17376 [==============================] - 0s 17us/step - loss: 0.0293 - acc: 0.9958 - val_loss: 1.0119 - val_acc: 0.9221
Epoch 7/10
17376/17376 [==============================] - 0s 15us/step - loss: 0.0239 - acc: 0.9967 - val_loss: 1.0101 - val_acc: 0.9217
Epoch 8/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0200 - acc: 0.9972 - val_loss: 1.0111 - val_acc: 0.9213
Epoch 9/10
17376/17376 [==============================] - 0s 18us/step - loss: 0.0156 - acc: 0.9975 - val_loss: 1.0175 - val_acc: 0.9211
Epoch 10/10
17376/17376 [==============================] - 0s 17us/step - loss: 0

Epoch 2/10
17376/17376 [==============================] - 0s 20us/step - loss: 0.0722 - acc: 0.9891 - val_loss: 0.9558 - val_acc: 0.9264

kappa improvement: 0.6872096687717173 (before: 0.6832984737377046), saving to exp3_img-text_mlp_model.hdf5
Epoch 3/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0537 - acc: 0.9926 - val_loss: 0.9771 - val_acc: 0.9254
Epoch 4/10
17376/17376 [==============================] - 0s 17us/step - loss: 0.0451 - acc: 0.9942 - val_loss: 0.9847 - val_acc: 0.9250
Epoch 5/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0341 - acc: 0.9955 - val_loss: 0.9972 - val_acc: 0.9238
Epoch 6/10
17376/17376 [==============================] - 0s 16us/step - loss: 0.0289 - acc: 0.9963 - val_loss: 0.9945 - val_acc: 0.9242
Epoch 7/10
17376/17376 [==============================] - 0s 17us/step - loss: 0.0228 - acc: 0.9972 - val_loss: 0.9904 - val_acc: 0.9229
Epoch 8/10
17376/17376 [==============================] - 0s 17us/step

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [212]:
for experiment in param_selection_results.keys():
    print(experiment + ": " + str(param_selection_results[experiment]))

300_0.1: 0.6852515810316551
300_0.2: 0.6871223351208229
300_0.3: 0.68686121506216
300_0.4: 0.689159652336207
300_0.5: 0.685658893687956
300_0.6: 0.6853433528813586
300_0.7: 0.6878746665438003
300_0.8: 0.6891409725882904
300_0.9: 0.6874184973695996
400_0.1: 0.6832988830279183
400_0.2: 0.6876135694830794
400_0.3: 0.6854449904544734
400_0.4: 0.6910287819300474
400_0.5: 0.6866319842816794
400_0.6: 0.6898778886492986
400_0.7: 0.6873365297429803
400_0.8: 0.6901676633495409
400_0.9: 0.6914310739270534
500_0.1: 0.6867188701608696
500_0.2: 0.6860627445487304
500_0.3: 0.6863893721984763
500_0.4: 0.686207355448785
500_0.5: 0.6903744622590591
500_0.6: 0.6877049657707486
500_0.7: 0.68878342194352
500_0.8: 0.6909000005464196
500_0.9: 0.6900227148787077


* 300_0.1: 0.6852515810316551
* 300_0.2: 0.6871223351208229
* 300_0.3: 0.68686121506216
* 300_0.4: 0.689159652336207
* 300_0.5: 0.685658893687956
* 300_0.6: 0.6853433528813586
* 300_0.7: 0.6878746665438003
* 300_0.8: 0.6891409725882904
* 300_0.9: 0.6874184973695996
* 400_0.1: 0.6832988830279183
* 400_0.2: 0.6876135694830794
* 400_0.3: 0.6854449904544734
* 400_0.4: 0.6910287819300474
* 400_0.5: 0.6866319842816794
* 400_0.6: 0.6898778886492986
* 400_0.7: 0.6873365297429803
* 400_0.8: 0.6901676633495409
* 400_0.9: 0.6914310739270534
* 500_0.1: 0.6867188701608696
* 500_0.2: 0.6860627445487304
* 500_0.3: 0.6863893721984763
* 500_0.4: 0.686207355448785
* 500_0.5: 0.6903744622590591
* 500_0.6: 0.6877049657707486
* 500_0.7: 0.68878342194352
* 500_0.8: 0.6909000005464196
* 500_0.9: 0.6900227148787077

In [ ]:
# Best setup: 400, 0.9
input_tp = Input(shape=sequence_x_2inputs_train[0][0].shape)
input_pp = Input(shape=sequence_x_2inputs_train[1][0].shape)
difference = subtract([input_pp, input_tp])
final_feat = concatenate([input_tp, input_pp, difference])
final_feat = Dense(400)(final_feat)
final_feat = LeakyReLU()(final_feat)
final_feat = Dropout(0.9)(final_feat)
model_output = Dense(1, activation = 'sigmoid')(final_feat)
combined_model = Model([input_tp, input_pp], model_output)
model_path = "exp3_img-text_mlp_model.hdf5"
checkpoint = ValidationCheckpoint(model_path, sequence_x_2inputs_test, data_test_y)
combined_model.compile(loss = 'binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
combined_model.fit(sequence_x_2inputs_train, data_train_y, validation_data = (sequence_x_2inputs_test, data_test_y), 
                   batch_size=4096, epochs=10, callbacks=[checkpoint])


# RNN sequencing

In [135]:
features_x_train = np.concatenate([text_features_train, image_features_train, lda_train_x[:,100:102]], axis = 1)
features_x_test = np.concatenate([text_features_test, image_features_test, lda_test_x[:,100:102]], axis = 1)
features_x_train.shape

(17376, 514)

In [137]:
bioes2Idx = {'B' : 0, 'I' : 1, 'E' : 2, 'S' : 3}

def binary_to_BIOES(sequence):
    x_all, y_all = zip(*sequence)
    # B begin
    # I Inside
    # O None - we dont have O ...
    # E End
    # S Single
    last_state = 'O'
    y_translated = []
    for y in y_all:
        if y == 1:
            if last_state == 'B':
                # single
                y_translated[-1] = 'S'
            elif last_state == 'I':
                # end
                y_translated[-1] = 'E'
            # begin
            y_translated.append('B')
        elif y == 0:
            # inside
            y_translated.append('I')
        else:
            raise ValueError('Only accept 0 or 1 as label.')
            
        last_state = y_translated[-1]
        
    if last_state == 'B':
        # single
        y_translated[-1] = 'S'
    elif last_state == 'I':
        y_translated[-1] = 'E'
        
    return [(x_all[i], bioes2Idx[y_translated[i]]) for i in range(len(x_all))]

In [138]:
from keras.preprocessing.sequence import pad_sequences

label2Idx = {'FirstPage':1, 'NextPage':0}

def create_sequences(data_instances, data_features, max_seq_len = 764):
    
    max_len = 0
    
    sequences = []
    prevBinder = ""
    tmp_sequence = []
    for i, instance in enumerate(data_instances):
        # "0 docid";"1 class";"2 type";"3 text";"4 binder"
        if prevBinder != instance[4]:
            if len(tmp_sequence) > 0:
                sequences.append(binary_to_BIOES(tmp_sequence))
            tmp_sequence = []
        tmp_sequence.append((data_features[i], label2Idx[instance[1]]))
        prevBinder = instance[4]
    if len(tmp_sequence) > 0:
        sequences.append(binary_to_BIOES(tmp_sequence))
        
        
        
    # create batches of same length
    batch_dict = {}
    for i, s in enumerate(sequences):
        if (len(s)) in batch_dict:
            batch_dict[len(s)].append(i)
        else:
            batch_dict[len(s)] = [i]
    batch_indexes = []
    for k in batch_dict.keys():
        batch_indexes.append(batch_dict[k])
        
    return batch_indexes, sequences

In [139]:
train_batch_idx, rnn_x_train = create_sequences(data_text_train, features_x_train)
test_batch_idx, rnn_x_test = create_sequences(data_text_test, features_x_test)

In [140]:
print(len(rnn_x_train))
print(len(rnn_x_train[20]))
print(len(rnn_x_train[20][1]))
print(len(test_batch_idx))
print(test_batch_idx)
print(rnn_x_train[20][0][0].shape[0])
for i in range(100):
    print(rnn_x_train[20][i][1])

76
275
2
24
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23]]
514
0
1
2
0
1
1
1
1
1
1
2
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
3
3
0
2
0
1
2
0
1
2
0
2
3
3
0
2
0
1
1
1
1
2
0
2
3
0
1
1
1
1
1
1
1
1
1
1
2
3
0
2
3
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
0
1
1
1
1
1
1
1


In [141]:
from keras.utils import Sequence, to_categorical
import math

class SequenceGenerator(Sequence):
    def __init__(self, sequence_data, batch_idx):
        self.sequence_data = sequence_data
        self.batch_idx = batch_idx

    def __len__(self):
        return len(self.batch_idx)

    def __getitem__(self, idx):
        inds = self.batch_idx[idx]
        batch_x, batch_y = self.process_sequence_data(inds)
        return batch_x, batch_y

    def process_sequence_data(self, inds):
        features = []
        output_labels = []
        for index in inds:
            tmp_features, tmp_output_labels = zip(*self.sequence_data[index])
            features.append(tmp_features)
            tmp_output_labels = to_categorical(tmp_output_labels, num_classes=4)
            output_labels.append(tmp_output_labels)
            
        batch_input = np.array(features)
        batch_output = np.array(output_labels)
        
        #print(batch_input.shape)
        #print(batch_output.shape)
        
        return (batch_input, batch_output)

In [142]:
class SequenceCheckpoint(Callback):
    def __init__(self, filepath, metric = 'kappa'):
        self.metric = metric
        self.max_metric = float('-inf')
        self.max_metrics = None
        self.filepath = filepath
        self.history = []
        #self.x = validation_x
        #self.x_batches = x_batches
        #self.validation_y = validation_y

    def on_epoch_end(self, epoch, logs={}):
        
        true_labels = []
        predicted_labels = []
        for i in range(len(rnn_x_test)):
            example_features, y_true_binder = zip(*rnn_x_test[i])
            example_features = np.array(example_features)
            example_features = np.reshape(example_features, (1,) + example_features.shape)
            # example_features.shape
            y_pred_binder = rnn_model.predict(example_features).argmax(axis=-1)
            predicted_labels.extend(y_pred_binder[0])
            true_labels.extend(y_true_binder)

            
        eval_metrics = {
            'accuracy' : sklm.accuracy_score(true_labels, predicted_labels),
            'f1_micro' : sklm.f1_score(true_labels, predicted_labels, average='micro'),
            'f1_macro' : sklm.f1_score(true_labels, predicted_labels, average='macro'),
            # 'f1_binary' : sklm.f1_score(true_labels, predicted_labels, average='binary', pos_label = 1),
            'kappa' : sklm.cohen_kappa_score(true_labels, predicted_labels)
        }
        eval_metric = eval_metrics[self.metric]
        self.history.append(eval_metric)
        
        if epoch > -1 and eval_metric > self.max_metric:
            print("\n" + self.metric + " improvement: " + str(eval_metric) + " (before: " + str(self.max_metric) + "), saving to " + self.filepath)
            self.max_metric = eval_metric     # optimization target
            self.max_metrics = eval_metrics   # all metrics
            self.model.save(self.filepath)

In [143]:
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy

In [160]:
rnn_input = Input(shape = (None,rnn_x_train[0][0][0].shape[0]))
rnn_layer = Bidirectional(GRU(200, return_sequences=True))(rnn_input)
# rnn_layer = Bidirectional(GRU(25, return_sequences=True))(rnn_layer)
rnn_dense = TimeDistributed(Dense(4))(rnn_layer)

crf = CRF(4, name = 'crf')
rnn_output = crf(rnn_dense)
rnn_model = Model(rnn_input, rnn_output)
rnn_model.compile(loss=crf_loss, optimizer='nadam', metrics=[crf_accuracy])
rnn_model.summary()

s_check = SequenceCheckpoint("exp3_seq_model.hdf5")

rnn_model.fit_generator(
    SequenceGenerator(rnn_x_train, train_batch_idx), 
    validation_data=SequenceGenerator(rnn_x_test, test_batch_idx), 
    callbacks=[s_check],
    epochs=15
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, None, 514)         0         
_________________________________________________________________
bidirectional_35 (Bidirectio (None, None, 50)          81000     
_________________________________________________________________
bidirectional_36 (Bidirectio (None, None, 50)          11400     
_________________________________________________________________
time_distributed_32 (TimeDis (None, None, 4)           204       
_________________________________________________________________
crf (CRF)                    (None, None, 4)           44        
Total params: 92,648
Trainable params: 92,648
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
71/71 [==============================] - 115s 2s/step - loss: 0.4228 - crf_accuracy: 0.8593 - val_loss: 0.6647 - val_crf_accura

KeyboardInterrupt: 

In [ ]:
rnn_model.load_weights(s_check.filepath)
# FirstPage = 1, NextPage = 0
idx2label = {0 : 1, 1 : 0, 2 : 0, 3 : 1}
y_true = []
y_pred = []
for i in range(len(rnn_x_test)):
    example_features, y_true_binder = zip(*rnn_x_test[i])
    example_features = np.array(example_features)
    example_features = np.reshape(example_features, (1,) + example_features.shape)
    # example_features.shape
    y_pred_binder = rnn_model.predict(example_features).argmax(axis=-1)
    y_pred.extend([idx2label[y] for y in y_pred_binder[0]])
    y_true.extend([idx2label[y] for y in y_true_binder])

In [123]:
sklm.cohen_kappa_score(y_true, y_pred)

0.6757859051984949

In [108]:
np.array([idx2label[y] for y in y_pred_binder[0]])

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [109]:
np.array([idx2label[y] for y in y_true_binder])

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,